### **Week 3,4: Function and Recursion** ###

#### 1. Example: Sound [(Video)](https://www.youtube.com/watch?v=TC_JcE42R2s&list=PL6BsET-8jgYVoDRPWXvw3q5ZsdpwVeEyY) ####

Why higher-order functions? We want to work in the space of functions! Take a look at the example below:  
**Example:** Write a .wav file. WAV format is simple but not compressed, not much used these days. We can aquisite .wav from nature or just generate wave signal.

![.wav File](../Sources/lec_06.png)

In [1]:
from wave import open
from struct import Struct
from math import floor

frame_rate = 11025

def encode(x):
    """
    Encode float x between -1 and 1 as 2 bytes.
    See library [struct]
    """
    i = int(16384*x)
    return Struct('h').pack(i)

def play(sampler, name='song.wav', seconds=2):
    """
    Write the output of a sampler function as a wav file.
    See library [wave]
    """
    out = open(name,'wb')
    out.setnchannels(1)
    out.setsampwidth(2)
    out.setframerate(frame_rate)
    t = 0

    while t < seconds*frame_rate:
        sample = sampler(t)
        out.writeframes(encode(sample))
        t = t+1
    out.close()

def tri(frequency, amplitude=0.3):
    """A continuous triangle wave."""
    period = frame_rate // frequency
    def sampler(t):
        saw_wave = t/period - floor(t/period + 0.5)
        tri_wave = 2*abs(2*saw_wave) - 1
        return amplitude*tri_wave
    return sampler

c_freq = 261.63


In [ ]:
# a demo to show triangle waves.
c = tri(c_freq)
t = 0
while t < 100:
    print(c(t))
    t +=1

play(c)

In [13]:
c_freq, e_freq, g_freq = 261.63, 329.63, 392.00

def both(f,g):
    return lambda t: f(t) + g(t)

play(both(tri(c_freq), tri(e_freq)))

def note(f, start, end, fade=0.01):
    def sampler(t):
        seconds = t/frame_rate
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        elif seconds < start + fade:
            return (seconds-start) / fade * f(t)
        elif seconds > end + fade:
            return (end - seconds) / fade * f(t)
        else:
            return f(t)
    return sampler

c, e = tri(c_freq), tri(e_freq)
play(both(note(c,0,1),note(e,1,2)))
g, low_g = tri(g_freq), tri(g_freq / 2)

### NOW THE TIME FOR MARIO!! ###
def mario_at(octave):
    c, e = tri(octave * c_freq), tri(octave * e_freq)
    g, low_g = tri(octave * g_freq), tri(octave * g_freq / 2)
    return mario(c,e,g,low_g)

def mario(c,e,g,low_g):
    z = 0
    song = note(e,z,z+1/8)
    z += 1/8
    song = both(song, note(e,z,z+1/8))
    z += 1/4
    song = both(song, note(e,z,z+1/8))
    z += 1/4
    song = both(song, note(c,z,z+1/8))
    z += 1/8
    song = both(song, note(e,z,z+1/8))
    z += 1/4
    song = both(song, note(g,z,z+1/4))
    z += 1/2
    song = both(song, note(low_g,z,z+1/8))
    z += 1/2
    return song

play(both(mario_at(2), mario_at(1)))

By the example of **sound**, we figure out that higher-order functions is a highly abstracted method of programming. We can apply this clever idea in practice.  
**AT LEAST** We can play a wonderful mario music!

#### 2. Function Abstraction [(Slide)](https://cs61a.org/assets/slides/07-Functional_Abstraction_1pp.pdf) ####

##### 2.1 Lambda Function Environment #####  

!["aha"](../Sources/lec_07.png)

In [4]:
# A good example to show frame structure: 
#   in <lambda y: a + y>, a=3 because it is not in the body of any function and is in global frame.
#   in <f(...)(a)>, a=3 because it is also in the global frame.
#   but in f(g), it returns 2*g(y) because in this local frame of f(g), a = 2

a = 3
def f(g):
    a = 2
    return lambda y: a * g(y)

f(lambda y: a + y)(a) # This always returns <a_in * (a_out + y)>, where y <- a_out


12

##### 2.2 Return  #####

`Return` is seen usually when a call expression is ended, and we would expect some 'output' from this call expression. Once `Return` happens nothing would be executed in this function.